Importanto Libs

In [1]:
import pandas as pd

Importando dados

In [6]:
def ler_arquivo(nome_arquivo):
    try: 
        return pd.read_excel(f'bases/{nome_arquivo}.xlsx')
    except FileNotFoundError as e: 
        return f'Erro: {e}'

In [4]:
df = ler_arquivo('dados de entregas')

SLA

In [25]:
# Total de entregas
df_total_entregas = df.loc[ df['status'] == 'ENTREGUE' ]

# Total de entregas sem atraso
sla_ok = df_total_entregas.loc[ df_total_entregas['dias_atraso'] == 0].shape[0]

# Percentual SLA
sla_percentual = (sla_ok / df_total_entregas.shape[0]) * 100

f'{round(sla_percentual, 1)}% das entregas foram feitas dentro do prazo.' 

'37.2% das entregas foram feitas dentro do prazo.'

KPI - Entregas atrasadas

In [26]:

percentual_atraso = (
    df_total_entregas.loc[ df_total_entregas['dias_atraso']  > 0].shape[0] /
    df_total_entregas.shape[0]
) * 100

round(percentual_atraso, 1)

62.8

SLA - Identificando onde as operações funcionam melhor

In [ ]:
sla_filial = df_total_entregas.groupby('filial')['dias_atraso'].mean()

round(sla_filial, 1)

In [ ]:
sla_filial = df_total_entregas.groupby('filial')['dias_atraso'].apply(
    lambda grupo: (grupo == 0).mean() * 100
).sort_values(ascending=False)

a = sla_filial.reset_index(name='% Entregas no Prazo')
round(a, 1)